# CanLII decision retriever


## Import libraries
The decision retrieval tool uses the following libraries:
* __Requests__
    * Allows the program to convert a shortened CanLII URL into a full CanLII URL
    * The full CanLII URL provides a directory structure for storying decisions locally
* __Validators__
    * Enables user input by providing URL verification
* __OS__
    * Lets the program save the retrieved decision locally

In [2]:
import requests
import validators
import os

## Validate the user's input
In production, the function takes a user's input and assigns it to ```url```. Once the user has provided a URL, that input will need to be checked to ensure it's a valid CanLII URL. This validation involves answering a few questions:
* Is the input a URL at all?
* Is the URL a CanLII URL?

Once the function verifies that the URL is a valid CanLII URL, it checks to see if the URL is either a shortened URL or contains query characters. If the URL is shortened, it retrieves the full version. If the URL contains query characters, it removes them.
* Is the CanLII URL shortened or full?
* Are there query characters appended to the URL?

### Verify that the input is a CanLII URL
The ```validators``` library includes a URL validator function. The ```verify_url``` function uses ```validators.url``` to check the user input. If the input is valid, it passes to the next check. If the input is invalid, the function calls for the user to provide new input and calls itself to re-validate it. 

Once the program verifies that the user inputted a URL, it next checks to see whether the URL is a CanLII URL. It does so by checking to see whether the URL contains the strings ```'https://canlii.ca'``` or ```'https://www.canlii.org'```.

In [8]:
def verify_url(url: str = "https://canlii.ca/t/jnnxk")->str:
    if validators.url(url) == True:
        print("* Valid URL")
    else:
        print("* Invalid URL")
        url = input("\t* Enter a valid URL: ")
        verify_url(url)
    if "https://canlii.ca" in url or "https://www.canlii.org" in url:
        print("* CanlII URL")
        return url
    else:
        print("* Not a CanLII URL.")
        url = input("\t* Enter a CanLII URL: ")
        verify_url(url)

### Resolve the URL into a path
* Checks to see if the URL is a shortened URL, and retrieves the full version if so
    * __NOTE__: currently experiencing 429 timeouts
* Checks for query characters and removes them if present
* Splits the URL into its components
* Returns the jurisdiction, court, and year components as a path to a directory

In [17]:
def resolve_url(url: str)->tuple:
    '''
    CanLII URLs use the following structure:
    
        {scheme}{separator}{language}/{jurisdiction}/{court}/{type}/{year}/
        {citation}/{file}
        
    This function returns a path where CanLII decisions can be saved, sorted by
    jurisdiction-court-year
    '''
    
    # Retrieves the full URL if the user provides the shortened form
    if "canlii.ca" in url:
        print("Shortened URL")
        url = requests.get(url)
        if requests.get(url) == 429:
            print("Request timeout")
            return
        elif requests.get(url) == 404:
            print("* Requested file doesn't exist")
            return
        else:
            url = requests.get(url).url
            print("* URL: " + url)

    # Removes query characters, if present
    if "?" in url:
        print("* Query characters detected")
        url = url.split('?')[0]
        print("\t* Removed")
        print("* URL: " + url)
    else:
        print("* URL: " + url)
    # Splits the URL into components
    split_url = url.split('/')

    # Exports a path based on the jurisdiction, court, and year
    jurisdiction = split_url[4]
    court = split_url[5]
    year = split_url[7]
    path =  f"canlii_data/{jurisdiction}/{court}/{year}"
    file_name = split_url[-1]
    print(f"* Path: {path}/{file_name}")
    
    return file_name, path, url

## Download decision

In [11]:
def download_decision(file_name: str, path: str, url: str)->str:
    if file_name is None or path is None:
        raise Exception("Input type is None")
        return
    if os.path.exists(f"{path}/{file_name}"):
        print("* WARNING: File already exists")
        return
    else:
        file_path = f"{path}/{file_name}"

        # Get the HTML of the decision
        response = requests.get(url)
        html = response.text

        # Write the HTML to a file
        with open(file_path, 'w') as f:
            f.write(html)
        print(f"* Saved as {path}/{file_name}")
        # return file_path

# Execute the functions

In [18]:
def download_from_user_url():
    url = input("Enter a CanLII URL: ")
    verified_url = verify_url(url)
    decision = resolve_url(verified_url)
    if decision is None:
        return
    else:
        download_decision(decision[0], decision[1], decision[2])

download_from_user_url()

Enter a CanLII URL:  https://www.canlii.org/en/ca/scc/doc/1990/1990canlii142/1990canlii142.html


* Valid URL
* CanlII URL
* URL: https://www.canlii.org/en/ca/scc/doc/1990/1990canlii142/1990canlii142.html
* Path: canlii_data/ca/scc/1990/1990canlii142.html
* Saved as canlii_data/ca/scc/1990/1990canlii142.html
